In [150]:
import numpy as np
import pandas as pd

In [151]:
data = pd.read_csv("./data/ANN 재공현황.csv")
data.head()

,BUS_CD,COIL_NO,LOT_NO,BASIC_COIL_NO,HOTCOI_NO,MOTCOI_NO,TRK_NO,EMG_CD,AFTEMG_CD,PKG_NO,...,CRT_DT,CRTCHR_NO,UPD_DT,UPDCHR_NO,COILEND_DT,SEMIPKG_YN,SEMIPKG_CD,PKG_CD,LEV2SE_YN,ZON_CD
0,1173DK,07KC1022111A,2007KC1022,07KC1022111,92311302,07KC1022111,NaN,1282N,NaN,NaN,...,2007-03-06 0:00,MGTEST,2019-07-12 14:50,이승용,NaN,NaN,NaN,NaN,E,1510P
1,1173DK,07KC1022211A,2007KC1022,07KC1022211,92311302,07KC1022211,NaN,1282N,NaN,NaN,...,2007-03-07 0:00,MGTEST,2013-01-16 14:26,김덕림,NaN,NaN,NaN,NaN,E,1510P
2,1173DK,07KC1022311A,2007KC1022,07KC1022311,92311302,07KC1022311,NaN,1282N,NaN,NaN,...,2007-03-07 0:00,MGTEST,2020-03-18 22:52,김백수,NaN,NaN,NaN,NaN,E,1510P
3,1173DK,07KK1D22111,2007KK1D22,07KK1D22111,HAT052900,NaN,NaN,NaN,NaN,NaN,...,2011-09-29 9:14,강연철,2021-02-09 10:08,이승용,NaN,NaN,NaN,1055A,E,1510P
4,1173DK,09KA1B77311,2009KA1B77,09KA1B77311,HAN026620,NaN,NaN,1282N,NaN,NaN,...,2010-01-01 0:00,MGTEST,2011-05-30 22:34,김도경,NaN,NaN,NaN,NaN,E,1510P


## 재품규격, ANN차수, 긴급 유무, 강종 기준으로 Grouping

### Merge할 Dataframe

In [152]:
data_copy = data.copy()

df = data_copy[["COIL_NO","PRDSPC_ID", "PNSPRC_CD", "WDT_LEN","WGT_WGT","IND_CD","OUD_LEN","STL_CD","EMG_CD"]]
df

,COIL_NO,PRDSPC_ID,PNSPRC_CD,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD
0,07KC1022111A,SCP1-H-B,AN10,392.0,5688,1260508,1616,10021,1282N
1,07KC1022211A,SCP1-H-B,AN10,392.0,5688,1260508,1616,10021,1282N
2,07KC1022311A,SCP1-H-B,AN10,392.0,5688,1260508,1616,10021,1282N
3,07KK1D22111,SCP1-H-B,AN10,1228.0,17479,1260400,1571,10021,NaN
4,09KA1B77311,SCP1-H-B,AN11,309.0,4811,1260508,1668,10021,1282N
...,...,...,...,...,...,...,...,...,...
390,22J120212111A,SAE1055M-R,AN10,368.0,5556,1260508,1645,10022,1282N
391,22J120213111A,SAE1055M-R,AN10,317.0,4786,1260508,1645,10022,1282N
392,22J12072111,SAE1070M-S,AN10,1240.0,21250,1260762,1833,10022,1282N
393,22J13014111,100CR6,AN10,1090.0,21470,1260762,1943,10023,1282N


1282Y, 1282S : 긴급

### 결측치 확인

In [153]:
df.isnull().sum()

COIL_NO      0
PRDSPC_ID    0
PNSPRC_CD    0
WDT_LEN      0
WGT_WGT      0
IND_CD       0
OUD_LEN      0
STL_CD       0
EMG_CD       1
dtype: int64

### 긴급 Column이 Nan인 결측치는 128N으로 채우기

In [154]:
df['EMG_CD'].fillna('1282N', inplace = True)

C:\Users\USER\AppData\Local\Temp\ipykernel_11708\887883402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EMG_CD'].fillna('1282N', inplace = True)


### 결측치 처리 확인

In [155]:
df.isnull().sum()

COIL_NO      0
PRDSPC_ID    0
PNSPRC_CD    0
WDT_LEN      0
WGT_WGT      0
IND_CD       0
OUD_LEN      0
STL_CD       0
EMG_CD       0
dtype: int64

### 긴급 수 확인

In [156]:
df["EMG_CD"].value_counts()

1282N    329
1282Y     48
1282S     18
Name: EMG_CD, dtype: int64

### special_cycle_test_data.csv 파일 불러오기
### **특별 cycle의 제품 규격 코드가 동일한 경우가 존재하여 엔에스하이텍(가장 하단에 위치) 고객사 제외하였음**

In [157]:
special_test = pd.read_csv("./data/special_cycle_test_data_csv.csv")
special_test_copy = special_test.fillna("-")
special_test_copy.rename(columns={ special_test_copy.columns[1] : 'PRDSPC_ID',
                                    special_test_copy.columns[2] : 'AN10',
                                    special_test_copy.columns[3] : 'AN11',
                                    special_test_copy.columns[4] : 'AN12',
                                    special_test_copy.columns[5] : 'AN13',}, inplace=True)
special_test_copy.set_index(['고객사','PRDSPC_ID'],inplace=True)
special_test_copy.head()

,,AN10,AN11,AN12,AN13
고객사,PRDSPC_ID,,,,
한국보그워너티에스,DK800,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용
UNIT TIDC INDIA,S700MC,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용
FAURECIA(CHINA) / BROSE CHINA,H800LA,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용,작업지시서 소둔 Cycle 적용
DYNAX CORP.,S55C-PA,-,620,620,620
한국보그워너티에스,S45CM-R,-,620,620,620


### Merge할 Dataframe

In [158]:
special_cycle_for_merge = pd.DataFrame(special_test_copy.stack()).reset_index()
special_cycle_for_merge.rename(columns={'level_2' : 'PNSPRC_CD', 0 : 'cycle'}, inplace=True)
special_cycle_for_merge

,고객사,PRDSPC_ID,PNSPRC_CD,cycle
0,한국보그워너티에스,DK800,AN10,작업지시서 소둔 Cycle 적용
1,한국보그워너티에스,DK800,AN11,작업지시서 소둔 Cycle 적용
2,한국보그워너티에스,DK800,AN12,작업지시서 소둔 Cycle 적용
3,한국보그워너티에스,DK800,AN13,작업지시서 소둔 Cycle 적용
4,UNIT TIDC INDIA,S700MC,AN10,작업지시서 소둔 Cycle 적용
...,...,...,...,...
111,(주)아스픽 / (주)엠알인프라오토,AISI4140M-S,AN13,750
112,유신정밀공업(주) / (주)화진금속,50CRV4M-S,AN10,750
113,유신정밀공업(주) / (주)화진금속,50CRV4M-S,AN11,750
114,유신정밀공업(주) / (주)화진금속,50CRV4M-S,AN12,750


## sorted_by_ANN, special_cycle_for_merge Merge하기

In [159]:
merged_df= pd.merge(df, special_cycle_for_merge, on=['PRDSPC_ID', 'PNSPRC_CD'], how='left')
#merged_data = pd.merge(sorted_by_ANN, special_cycle_test_copy, left_on='PRDSPC_ID', right_on='제품 규격 코드',how='left')
merged_df = merged_df[["COIL_NO","PRDSPC_ID", "PNSPRC_CD", "cycle", "WDT_LEN","WGT_WGT","IND_CD","OUD_LEN","STL_CD","EMG_CD","고객사"]]
merged_df

,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사
0,07KC1022111A,SCP1-H-B,AN10,NaN,392.0,5688,1260508,1616,10021,1282N,NaN
1,07KC1022211A,SCP1-H-B,AN10,NaN,392.0,5688,1260508,1616,10021,1282N,NaN
2,07KC1022311A,SCP1-H-B,AN10,NaN,392.0,5688,1260508,1616,10021,1282N,NaN
3,07KK1D22111,SCP1-H-B,AN10,NaN,1228.0,17479,1260400,1571,10021,1282N,NaN
4,09KA1B77311,SCP1-H-B,AN11,NaN,309.0,4811,1260508,1668,10021,1282N,NaN
...,...,...,...,...,...,...,...,...,...,...,...
390,22J120212111A,SAE1055M-R,AN10,NaN,368.0,5556,1260508,1645,10022,1282N,NaN
391,22J120213111A,SAE1055M-R,AN10,NaN,317.0,4786,1260508,1645,10022,1282N,NaN
392,22J12072111,SAE1070M-S,AN10,NaN,1240.0,21250,1260762,1833,10022,1282N,NaN
393,22J13014111,100CR6,AN10,750*Max,1090.0,21470,1260762,1943,10023,1282N,(주)지에스앤티 / 한국엔에스케이


In [160]:
merged_df.isnull().sum()

COIL_NO        0
PRDSPC_ID      0
PNSPRC_CD      0
cycle        307
WDT_LEN        0
WGT_WGT        0
IND_CD         0
OUD_LEN        0
STL_CD         0
EMG_CD         0
고객사          307
dtype: int64

### ANN 차수와 강종에 따라 구분

### 10021 : 일반강
- ANN 0차 : X
- ANN 1차 이상 : 620

### 10022 : SC강
- ANN 0차 : 725
- ANN 1차 이상 : 700

### 10023 : SK강
- ANN 0차 : 735
- ANN 1차 이상 : 735


In [161]:
merged_df[(merged_df['PNSPRC_CD'] == 'AN10') & (merged_df['STL_CD'] == 10021)]

,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사
0,07KC1022111A,SCP1-H-B,AN10,NaN,392.0,5688,1260508,1616,10021,1282N,NaN
1,07KC1022211A,SCP1-H-B,AN10,NaN,392.0,5688,1260508,1616,10021,1282N,NaN
2,07KC1022311A,SCP1-H-B,AN10,NaN,392.0,5688,1260508,1616,10021,1282N,NaN
3,07KK1D22111,SCP1-H-B,AN10,NaN,1228.0,17479,1260400,1571,10021,1282N,NaN


In [162]:
merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] == 'AN10') & (merged_df['STL_CD'] == 10021) & (merged_df['cycle'].isnull()), "-", merged_df['cycle'])
merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] != 'AN10') & (merged_df['STL_CD'] == 10021) & (merged_df['cycle'].isnull()), "620", merged_df['cycle'])

merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] == 'AN10') & (merged_df['STL_CD'] == 10022) & (merged_df['cycle'].isnull()), "725", merged_df['cycle'])
merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] != 'AN10') & (merged_df['STL_CD'] == 10022) & (merged_df['cycle'].isnull()), "700", merged_df['cycle'])

merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] == 'AN10') & (merged_df['STL_CD'] == 10023) & (merged_df['cycle'].isnull()), "735", merged_df['cycle'])
merged_df['cycle'] = np.where((merged_df['PNSPRC_CD'] != 'AN10') & (merged_df['STL_CD'] == 10023) & (merged_df['cycle'].isnull()), "735", merged_df['cycle'])

In [163]:
merged_df.isnull().sum()

COIL_NO        0
PRDSPC_ID      0
PNSPRC_CD      0
cycle          0
WDT_LEN        0
WGT_WGT        0
IND_CD         0
OUD_LEN        0
STL_CD         0
EMG_CD         0
고객사          307
dtype: int64

In [164]:
merged_df[(merged_df['PNSPRC_CD'] == 'AN10') & (merged_df['STL_CD'] == 10021)]

,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사
0,07KC1022111A,SCP1-H-B,AN10,-,392.0,5688,1260508,1616,10021,1282N,NaN
1,07KC1022211A,SCP1-H-B,AN10,-,392.0,5688,1260508,1616,10021,1282N,NaN
2,07KC1022311A,SCP1-H-B,AN10,-,392.0,5688,1260508,1616,10021,1282N,NaN
3,07KK1D22111,SCP1-H-B,AN10,-,1228.0,17479,1260400,1571,10021,1282N,NaN


In [165]:
drop_column_index = merged_df[merged_df["cycle"].str.contains("-")].index
merged_df.drop(drop_column_index, inplace=True)
merged_df

,COIL_NO,PRDSPC_ID,PNSPRC_CD,cycle,WDT_LEN,WGT_WGT,IND_CD,OUD_LEN,STL_CD,EMG_CD,고객사
4,09KA1B77311,SCP1-H-B,AN11,620,309.0,4811,1260508,1668,10021,1282N,NaN
5,21M12B721111A,S45CM-S,AN11,700,157.0,1376,1260400,1290,10022,1282N,NaN
6,21M12D061111A,50CRV4M-S,AN10,750,248.0,2091,1260508,1275,10022,1282S,유신정밀공업(주) / (주)화진금속
7,22A122851111A,S55CM-R,AN10,725,410.0,5121,1260508,1511,10022,1282N,NaN
8,22A122852111A,S55CM-R,AN10,725,410.0,5121,1260508,1511,10022,1282N,NaN
...,...,...,...,...,...,...,...,...,...,...,...
390,22J120212111A,SAE1055M-R,AN10,725,368.0,5556,1260508,1645,10022,1282N,NaN
391,22J120213111A,SAE1055M-R,AN10,725,317.0,4786,1260508,1645,10022,1282N,NaN
392,22J12072111,SAE1070M-S,AN10,725,1240.0,21250,1260762,1833,10022,1282N,NaN
393,22J13014111,100CR6,AN10,750*Max,1090.0,21470,1260762,1943,10023,1282N,(주)지에스앤티 / 한국엔에스케이


In [170]:
merged_df.to_csv("./data/data_df.csv", sep=",", na_rep = "NaN")